In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import datasets, surface, plotting

In [2]:
def filter_r(r, p, fill=None, crit=1e-3):
    # p = surface.vol_to_surf(p, fsaverage[f'pial_{hemi}'])
    
    if fill is None:
        i = np.where(p >= crit)
        r[i] = 0
    else:
        i = np.where(p < crit)
        r[i] = fill
    return r

In [3]:
def mkNifti(mask, name, im, zeros=True):
    if zeros:
        out_im = np.zeros(mask.size)
    else:
        out_im = np.ones(mask.size)
    inds = np.where(mask)[0]
    out_im[inds] = np.load(name)
    return nib.Nifti1Image(out_im.reshape(im.shape), affine=im.affine)

In [4]:
process = 'PlotFeatures'
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis'
data_dir = f'{top_dir}/data/raw'
out_dir = f'{top_dir}/data/interim'
sid='03'
figure_dir = f'{top_dir}/reports/figures/{process}/sub-{sid}'
if not os.path.exists(figure_dir):
    os.mkdir(figure_dir)
    os.mkdir(f'{figure_dir}/feature')
    os.mkdir(f'{figure_dir}/feature/glass')
    os.mkdir(f'{figure_dir}/feature/surf')
    os.mkdir(f'{figure_dir}/category')
    os.mkdir(f'{figure_dir}/category/glass')
    os.mkdir(f'{figure_dir}/category/surf')
fsaverage = datasets.fetch_surf_fsaverage()

In [5]:
df = pd.read_csv(f'{data_dir}/annotations/annotations.csv')
train = pd.read_csv(f'{data_dir}/annotations/train.csv')
df = df.merge(train)
df.sort_values(by=['video_name'], inplace=True)
features = df.columns.to_list()
features.remove('video_name')

In [6]:
features_cat = ['visual', 'visual', 'visual',
                'visual', 'visual', 'detection',
                'detection', 'features', 'features',
                'features', 'features', 'features']

# features_cat = [1, 1, 1, 1, 1,
#                 2, 2, 
#                 3, 3, 3, 3, 3]

In [7]:
mask = np.load(f'{out_dir}/group_reliability/sub-all_reliability-mask.npy')
mask_im = nib.load(f'{out_dir}/group_reliability/sub-all_stat-rho_statmap.nii.gz')

name = f'{out_dir}/VoxelPermutation/sub-{sid}/sub-{sid}_feature-indoor_ps.npy'
ps = mkNifti(mask, name, mask_im, zeros=False)
# n = surface.vol_to_surf(ps, fsaverage['pial_right']).shape
ps = np.array(ps.dataobj)
# texture = {'left': np.zeros_like(ps), 'right': np.zeros_like(ps)}
# texture = {'left': filter_r(texture['left'], ps, hemi='left', fill=1),
#            'right': filter_r(texture['right'], ps, hemi='right', fill=1)}

In [8]:
def fill_val(texture, out_dir, sid, features, fill=1):
    for feature in features:
        name = f'{out_dir}/VoxelPermutation/sub-{sid}/sub-{sid}_feature-{feature}_ps.npy'
        ps = mkNifti(mask, name, mask_im, zeros=False)
        ps = np.array(ps.dataobj)
        texture = filter_r(texture, ps, fill=fill)
    return nib.Nifti1Image(texture, affine=mask_im.affine)

In [9]:
    # plotting.view_img(texture, cmap=cmap, threshold=0.9,
    #                   symmetric_cmap=False, vmax=1.2,
    #                   resampling_interpolation='nearest')

In [10]:
for feature in features:
    texture = fill_val(np.zeros_like(ps, dtype='int'), out_dir, sid, [feature])
    plotting.plot_img_on_surf(texture, inflate=True,
                              threshold=0.1, colorbar=True,
                              views=['lateral', 'medial', 'ventral'],
                              output_file=f'{figure_dir}/feature/surf/feature-{feature}_view-surf.png')
    plotting.plot_glass_brain(texture,
                              output_file=f'{figure_dir}/feature/glass/feature-{feature}_view-glass.png')

In [ ]:
for category in np.unique(np.array(features_cat)):
    filtered_features = [feature for feature, cat in zip(features, features_cat) if cat == category]
    print(filtered_features)
    texture = fill_val(np.zeros_like(ps, dtype='int'),
                       out_dir, sid, filtered_features)
    
    plotting.plot_img_on_surf(texture, inflate=True,
                              threshold=0.1, colorbar=True,
                              views=['lateral', 'medial', 'ventral'],
                              output_file=f'{figure_dir}/category/surf/category-{category}_view-surf.png')
    plotting.plot_glass_brain(texture,
                              output_file=f'{figure_dir}/category/glass/category-{category}_view-glass.png')

['joint action', 'communication']
['cooperation', 'dominance', 'intimacy', 'valence', 'arousal']


In [ ]:
# class mkcmap():
#     def __init__(self, val):
#         self.colors = val
        
#     def __call__(self, X):
#         if (type(X) is not list) and (type(X) is not np.ndarray):
#             return self.colors[int(X)]
#         else:
#             out = []
#             for x in X:
#                 out.append(self.colors[int(x)])
#             return out

# # colors = ['#48D4E1', '#8C55FD', '#FADC00']
# colors = [(72, 212, 225, 1),
#           (140, 85, 253, 1), 
#           (250, 220, 0, 1)]
# cmap = mkcmap(colors)
# cmap(X)